In [1]:
import cv2 as cv
import torch
from PIL import Image
from facenet_pytorch import MTCNN
from deepface import DeepFace
import numpy as np
import os

d:\Nam_5\KLTN\e-kyc\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# Extract face using MTCNN
def extract_face(img: np.ndarray, model: MTCNN, padding=None, min_prob=0.9):
    boxes, prob, landmarks = model.detect(img, landmarks=True)

    if boxes is not None:
        boxes = boxes[prob > min_prob]

        max_area = 0
        max_box = [0, 0, 0, 0]
        max_landmarks = []

        for i, box in enumerate(boxes):
            box = np.clip(box, 0, np.inf).astype(np.uint32)
            x1, y1, x2, y2 = box
            if (x2 - x1) * (y2 - y1) > max_area:
                max_box = padding_face(box, padding)
                max_area = (x2 - x1) * (y2 - y1)
                max_landmarks = landmarks[i]

        x1, y1, x2, y2 = max_box
        face = img[y1: y2, x1: x2, ...]

        return face, max_box, max_landmarks
    else:
        return img, None, None

# Padding function for bounding box
def padding_face(box: np.ndarray, padding=None):
    x1, y1, x2, y2 = box
    cx = (x1 + x2) // 2
    cy = (y1 + y2) // 2
    w = x2 - x1
    h = y2 - y1
    if padding:
        if isinstance(padding, float):
            w = w * padding
            h = h * padding
        else:
            w = w + padding
            h = h + padding

    x1 = cx - w // 2
    x2 = cx + w // 2
    y1 = cy - h // 2
    y2 = cy + h // 2

    box = np.clip([x1, y1, x2, y2], 0, np.inf).astype(np.uint32)
    return box

# Helper function to load images
def get_image(filename):
    img = cv.imread(filename)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    return img

# Function to verify two images using DeepFace.verify()
def verify(img1: np.ndarray, img2: np.ndarray, detector_model: MTCNN):
    # Step 1: Extract faces using MTCNN
    face1, _, _ = extract_face(img1, detector_model)
    face2, _, _ = extract_face(img2, detector_model)

    # Step 2: Save the cropped faces to the 'crop' folder
    save_path1 = "crop/face1.jpg"
    save_path2 = "crop/face2.jpg"
    save_cropped_face(face1, save_path1)
    save_cropped_face(face2, save_path2)

    # Step 3: Use DeepFace to verify the faces
    result = DeepFace.verify(
        img1_path=save_path1,
        img2_path=save_path2,
        model_name="Facenet",  # Can be changed to other models like VGG-Face, OpenFace, etc.
        enforce_detection=False,  # Since we're using MTCNN, no need to enforce detection here
        distance_metric="cosine"  # Or other metrics like "euclidean", "L1"
    )
    
    return result



In [10]:
# Save the cropped face to a folder named 'crop'
def save_cropped_face(face: np.ndarray, save_path: str):
    if not os.path.exists('crop'):
        os.makedirs('crop')  # Create 'crop' directory if it doesn't exist
    cv.imwrite(save_path, cv.cvtColor(face, cv.COLOR_RGB2BGR))  # Save the image in BGR format

In [12]:
# Loading images using OpenCV
filename1 = "./file_test/ccDat.jpg"
filename2 = "./file_test/datv2.jpg"
image1 = get_image(filename1)
image2 = get_image(filename2)

# Device setup for MTCNN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
detector_model = MTCNN(device=device)

# Verifying the images using DeepFace
results = verify(image1, image2, detector_model)
print(results)

{'verified': True, 'distance': 0.2420265020039819, 'threshold': 0.4, 'model': 'Facenet', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 0, 'y': 23, 'w': 361, 'h': 422, 'left_eye': (278, 183), 'right_eye': (113, 190)}, 'img2': {'x': 0, 'y': 39, 'w': 777, 'h': 954, 'left_eye': (569, 418), 'right_eye': (238, 414)}}, 'time': 2.08}


In [13]:
# Function to verify an image against a real-time frame
def verify_with_camera(img1: np.ndarray, detector_model: MTCNN):
    cap = cv.VideoCapture(0)  # Open webcam
    if not cap.isOpened():
        print("Error: Cannot access the webcam.")
        return

    print("Press 'q' to quit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to read from the webcam.")
            break

        # Convert the frame to RGB
        frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

        # Extract face from the webcam frame
        face_cam, _, _ = extract_face(frame_rgb, detector_model)

        if face_cam is not None:
            # Save the webcam face to 'crop' folder
            save_path_cam = "crop/face_cam.jpg"
            save_cropped_face(face_cam, save_path_cam)

            # Verify the first image against the webcam face
            result = DeepFace.verify(
                img1_path="crop/face1.jpg",  # Pre-cropped first image
                img2_path=save_path_cam,
                model_name="Facenet",
                enforce_detection=False,
                distance_metric="cosine"
            )

            print(result)  # Display the verification result

        # Display the webcam feed
        cv.imshow("Webcam", frame)

        # Exit the loop when 'q' is pressed
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv.destroyAllWindows()
    
    

In [14]:
# Loading the first image
filename1 = "./file_test/ccDat.jpg"
image1 = get_image(filename1)

# Device setup for MTCNN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
detector_model = MTCNN(device=device)

# Extract and save the face from the first image
face1, _, _ = extract_face(image1, detector_model)
if face1 is not None:
    save_cropped_face(face1, "crop/face1.jpg")
else:
    print("Error: No face detected in the first image.")
    exit()

# Start real-time verification
verify_with_camera(image1, detector_model)

Press 'q' to quit.
{'verified': False, 'distance': 0.4274185999408249, 'threshold': 0.4, 'model': 'Facenet', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 0, 'y': 23, 'w': 361, 'h': 422, 'left_eye': (278, 183), 'right_eye': (113, 190)}, 'img2': {'x': 0, 'y': 0, 'w': 177, 'h': 227, 'left_eye': None, 'right_eye': None}}, 'time': 1.19}
{'verified': False, 'distance': 0.44778536765056665, 'threshold': 0.4, 'model': 'Facenet', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 0, 'y': 23, 'w': 361, 'h': 422, 'left_eye': (278, 183), 'right_eye': (113, 190)}, 'img2': {'x': 0, 'y': 0, 'w': 173, 'h': 217, 'left_eye': None, 'right_eye': None}}, 'time': 1.42}
{'verified': False, 'distance': 0.40046587495467745, 'threshold': 0.4, 'model': 'Facenet', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 0, 'y': 23, 'w': 361, 'h': 422, 'left_eye': (278, 183), 'right_eye': (113